In [1]:
#!pip install opencv-python
import pandas as pd
import keras
import cv2

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from cutmix_keras import CutMixImageDataGenerator  
import random
def random_brightness(image):
    # Convert 2 HSV colorspace from RGB colorspace
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Generate new random brightness
    rand = random.uniform(0.3, 1.0)
    hsv[:, :, 2] = rand*hsv[:, :, 2]
    # Convert back to RGB colorspace
    new_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return new_img
data_gen_args = {'channel_shift_intencity':0.9,'brightness':0.87}
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function = random_brightness, brightness_range = [0.6,0.897],horizontal_flip=True,validation_split=0.25)

train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(152,152),
        batch_size=72,
        class_mode='binary',subset='training', shuffle = True)

validation_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(152,152),
        batch_size=72,
        class_mode='binary', subset='validation')


Found 813425 images belonging to 2 classes.
Found 271141 images belonging to 2 classes.


In [3]:
train_generator.class_indices

{'0': 0, '1': 1}

In [3]:
import tensorflow as tf
base = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(200,200,3), pooling=None, classes=2)


In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense,MaxPooling2D, dot, Flatten, Dropout, concatenate, LSTM, TimeDistributed, Lambda
inp = Input(shape=(200,200,3))
#x = MaxPooling2D()(base.output)
x = Flatten()(inp)
#x = TimeDistributed(base)(inp)
#x = keras.layers.GaussianNoise(0.2)(x)
#x = LSTM(128)(x)
x = Dense(128, activation='relu')(x)
#x = keras.layers.GaussianNoise(0.1)(x)
x = Dropout(rate=0.1)(x)
x = Dense(64, activation='elu')(x)
#x = Dropout(rate=0.2)(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[inp], outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
flatten (Flatten)            (None, 120000)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               15360128  
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 15,368,449
Trainable params: 15,368,449
Non-trainable params: 0
_________________________________________________

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense,MaxPooling2D, dot, Flatten, Dropout, concatenate, LSTM, TimeDistributed, Lambda
from tensorflow.keras.models import Model
from keras_radam import RAdam
from deepface import create_deepface, get_weights
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow 
base = create_deepface()
weights = get_weights()
base.load_weights(weights)
base.trainable = False
x = Dense(1024, activation='relu')(base.output)
x = Dropout(rate=0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[base.input], outputs=output)

#model.summary()

#checkpoint = ModelCheckpoint(r"D:\Projects\best_model.hdf5", monitor='loss', verbose=1,
#    save_best_only=True, mode='auto', period=1)
model.compile(tensorflow.keras.optimizers.Adam(lr=1e-3),loss=tensorflow.keras.losses.binary_crossentropy,
    metrics=[tensorflow.metrics.binary_accuracy]
)

model.fit(train_generator,epochs=5,validation_data=validation_generator,use_multiprocessing=False)

Model: "DeepFace"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
C1 (Conv2D)                  (None, 142, 142, 32)      11648     
_________________________________________________________________
M2 (MaxPooling2D)            (None, 71, 71, 32)        0         
_________________________________________________________________
C3 (Conv2D)                  (None, 63, 63, 16)        41488     
_________________________________________________________________
L4 (LocallyConnected2D)      (None, 55, 55, 16)        62774800  
_________________________________________________________________
L5 (LocallyConnected2D)      (None, 25, 25, 16)        7850000   
_________________________________________________________________
L6 (LocallyConnected2D)      (None, 21, 21, 16)        2829456   
_________________________________________________________________
F0 (Flatten)                 (None, 7056)              0  

In [ ]:
model.save('try_1.h5')

NameError: name 'create_deepface' is not defined

In [ ]:
import Augmentor
p = Augmentor.Pipeline(r"G:\jetjo\train\REAL")

p.rotate(probability=0.4, max_left_rotation=10, max_right_rotation=10)
#p.flip_top_bottom(0.5)
p.crop_random(probability=0.8, percentage_area=0.95)
p.zoom_random(probability=0.8, percentage_area=0.95)
p.sample(410000)